In [3]:
import numpy as np
import regex as re

<regex.Match object; span=(6, 7), match='8'>

In [ ]:
class Data_Preprocessing:
    def __init__(self, x_data, y_data, x_lemmas):
        self.x = x_data
        self.y = y_data
        self.x_l = x_lemmas
        
    #b(x, i) in Intro to CRFs Hanna M. Wallach et al.
    def equivalence_func(self, x, i, x_seq):

        return 1 if x == x_seq[i] else 0

    def transition_func(self, y_prev, y, x, i, x_seq, y_seq):

#         transition_encodings = []

#         for y_prev in y_seq[:-1]:
#             for y in y_seq[1:]:
#                 transition_encodings += equivalence_func(x, i, x_seq) if y_prev == y_seq[i-1] and y == y_seq[i] else 0

#         return transition_encodings

    #     return [lambda y_prev, y, x, i, x_seq, y_seq: equivalence_func(x, i, x_seq) if y_prev == y_seq[i-1] and y == y_seq[i] else 0
    #            for y_prev in y_seq[:-1] for y in y_seq[1:]]
    
        return equivalence_func(x, i, x_seq) if y_prev == y_seq[i-1] and y == y_seq[i] else 0

    def potential_funcs(self, y, x, x_lemma, i):

        features = []
        
        #feature 1: 1 if lemma form is same as base form, else 0
        features += [lambda y_prev, self.y, self.x, i: 1 if self.x_l[i] is self.x[i] else 0]
        #feature 2: 1 if a numeral is present in x, else 0
        features += [lambda y_prev, self.y, self.x, i: 1 if re.search('[0-9]', self.x[i]) else 0]

        return features

    #Generates features for a single sentence in out x dataset
    def featurize_sentence(self, sentence, labels, funcs):

        feature_len = len(funcs)
        features = np.zeros(len(sentence) + 1, len(labels), len(labels), feature_len)
        return 0
#         for i in range(0, len(sentence) + 1):
#             for j in range(0, len(labels)):
#                 for k in range(0, len(labels)):
#                     for func in funcs: